In [ ]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import time, os
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Web Scraping: Selenium

In [ ]:
options = Options()
ua = UserAgent()
user_agent = ua.random
print(user_agent)
options.add_argument(user_agent)
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
web = "https://www.aspca.org/adopt-pet/adoptable-dogs-your-local-shelter#petfocus_0=&page_0=2&breed_0=&sex_0=&distance_0=100000&location_0=90703&action_0=search"

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(web)

### Testing scraping data

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
x=soup.find_all('img', class_="rgtkSearchPetPicImg")

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
y=soup.find_all('div', class_="rgtkSearchPetName")
str(y[0]).split(', ')[1]
#https://www.aspca.org/adopt-pet/adoptable-dogs-your-local-shelter#action_0=pet&animalID_0=16616977&petIndex_0=1

In [ ]:
driver.quit()

### For loop for scrapping all data

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
image_lst=[]
web_lst=[]

In [ ]:
for page in range(5,873):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    web = "https://www.aspca.org/adopt-pet/adoptable-dogs-your-local-shelter#petfocus_0=&page_0={}&breed_0=&sex_0=&distance_0=100000&location_0=90703&action_0=search".format(page)
    driver.get(web)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    x=soup.find_all('img', class_="rgtkSearchPetPicImg")
    y=soup.find_all('div', class_="rgtkSearchPetName")
    for i in range(len(x)):
        image_lst.append(str(x[i]).split('src="')[1].split('"')[0])
        web_lst.append("https://www.aspca.org/adopt-pet/adoptable-dogs-your-local-shelter#action_0=pet&animalID_0={}&petIndex_0=0".format(str(x[i]).split('src="')[1].split('"')[0].split('/')[8]))
    time.sleep(1)
    driver.quit()

In [ ]:
len(image_lst), len(web_lst)

### Save file

In [ ]:
image_lst_df.to_csv('pet_image.csv')
web_lst_df.to_csv('pet_web.csv')

In [ ]:
image_lst_df=pd.DataFrame(image_lst)
web_lst_df=pd.DataFrame(web_lst)

In [ ]:
image_web_lst_df=image_lst_df
#pd.concat([image_lst_df,web_lst_df])

In [ ]:
image_web_lst_df['web']=web_lst_df

In [ ]:
image_web_lst_df.columns=['image','web']

In [ ]:
image_web_lst_df.to_csv('pet_image_web.csv')

# Scrap for adoptable dog profile

In [ ]:
pet_image_web=pd.read_csv('pet_image_web.csv')

In [ ]:
pet_image_web.columns=['age', 'image', 'web']
pet_image_web['age']=0
pet_image_web['name']=0
pet_image_web['size']=0
pet_image_web['sex']=0

In [ ]:
options = Options()
ua = UserAgent()
user_agent = ua.random
print(user_agent)
options.add_argument(user_agent)
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
web = pet_image_web.web[1]

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(web)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
x=soup.find('div', class_="rgtkPetInfoIndented",id="rgtkPetInfoIndented_0")
x
2021-int(str(x).split('<li>Birth date: ')[1].split('/')[2].split(' ')[0])
#2021-int(str(x).split('<li>Birth date: ')[1].split('/')[2].split(' ')[0])

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
name=str(soup.find('div', class_="rgtkPetTitleName",id="rgtkPetTitleName_0")).split('>')[1].split('<')[0]
size=str(soup.find('span', class_="rgtkPetDetailsGeneralSizePotential",id="rgtkPetDetailsGeneralSizePotential_0")).split('>')[1].split('<')[0]
age=str(soup.find('span', class_="rgtkPetDetailsAge",id="rgtkPetDetailsAge_0")).split('>')[1].split('<')[0]
sex=str(soup.find('span', class_="rgtkPetDetailsSex",id="rgtkPetDetailsSex_0")).split('>')[1].split('<')[0]
name,size,age,sex


In [ ]:
for i, web in enumerate(pet_image_web.web[range(0,len(pet_image_web))]):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(web)
    time.sleep(1.5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        name=str(soup.find('div', class_="rgtkPetTitleName",id="rgtkPetTitleName_0")).split('>')[1].split('<')[0]
        pet_image_web.name[i]=name
    except:
        pet_image_web.name[i]=None
    
    try:
        size=str(soup.find('span', class_="rgtkPetDetailsGeneralSizePotential",id="rgtkPetDetailsGeneralSizePotential_0")).split('>')[1].split('<')[0]
        pet_image_web['size'][i]=size
    except:
        pet_image_web['size'][i]=None
    
    
    try:
        sex=str(soup.find('span', class_="rgtkPetDetailsSex",id="rgtkPetDetailsSex_0")).split('>')[1].split('<')[0]
        pet_image_web.sex[i]=sex
    except:
        pet_image_web.sex[i]=None
       
    
    try:
        age=str(soup.find('span', class_="rgtkPetDetailsAge",id="rgtkPetDetailsAge_0")).split('>')[1].split('<')[0]
        pet_image_web.age[i]=age
    except:
        pet_image_web.age[i]=None
        
    print(i, sex,age, size,name)
    #time.sleep(1)
    driver.quit()

In [ ]:
pet_image_web.to_csv('pet_image_web2.csv')

### Check nan cell and rerun for loop

In [ ]:
index_nan=pet_image_web.index[pd.isna(pet_image_web1['age'])]
index_nan

In [ ]:
pet_image_web.web[index_nan].index

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
for web, i in zip(pet_image_web.web[index_nan],index_nan):
    driver.get(web)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        name=str(soup.find('div', class_="rgtkPetTitleName",id="rgtkPetTitleName_0")).split('>')[1].split('<')[0]
        pet_image_web.name[i]=name
    except:
        pet_image_web.name[i]=None
    
    try:
        size=str(soup.find('span', class_="rgtkPetDetailsGeneralSizePotential",id="rgtkPetDetailsGeneralSizePotential_0")).split('>')[1].split('<')[0]
        pet_image_web['size'][i]=size
    except:
        pet_image_web['size'][i]=None
    
    
    try:
        sex=str(soup.find('span', class_="rgtkPetDetailsSex",id="rgtkPetDetailsSex_0")).split('>')[1].split('<')[0]
        pet_image_web.sex[i]=sex
    except:
        pet_image_web.sex[i]=None
       
    
    try:
        age=str(soup.find('span', class_="rgtkPetDetailsAge",id="rgtkPetDetailsAge_0")).split('>')[1].split('<')[0]
        pet_image_web.age[i]=age
    except:
        pet_image_web.age[i]=None
        
    print(i, web, sex,age, size,name)

driver.quit()